https://www.philipzucker.com/td4-4bit-cpu/





In [10]:
from collections import namedtuple
class TaggedUnion():
    def __init__(self, name, cases):
        self.name = name
        self.constructors = []
        for name, args in cases:
            cons = namedtuple(name, args)
            self.constructors.append(cons)
            setattr(self, name, lambda self, *args, **kwargs: cons(*args, **kwargs))
            setattr(self, "is_" + name, lambda x: isinstance(x, cons))
            for fname in args:
                setattr(self, fname, lambda x: getattr(x, fname))
    def num_constructors(self):
        return len(self.constructors)
    def constructor(self, idx):
        return self.constructors[idx]
    

AExpr = TaggedUnion("AExpr", [
    ("Int", ["value"]),
    ("Add", ["lhs", "rhs"]),
])
            


AExpr.Int(1,2,4)



Add(lhs=2, rhs=4)

In [ ]:
from collections import namedtuple
from enum import Enum

Op = Enum("Op", "ADD MOV IN OUT JNC JMP")
Op = Enum("Op", "ADD_A ADD_B MOV_AB MOV_BA IN_A IN_B OUT_B OUT JNC JMP")
Insn = namedtuple("Insn", "op im")
State = namedtuple("State", "a, b, out, pc, c, inp, program")
def step(state):
    a, b, out, pc, c, inp, program = state 
    insn = program[pc]
    im = insn.im
    match insn.op:
        case Op.ADD_A:
            im_int = int(im)
            assert 0 <= im_int < 16
            a_tmp = a+im_int
            return State(a_tmp%16, b, out, (pc+1)%16, a_tmp>=16, inp, program)
        case Op.MOV_A_B:
            return State._replace(pc = (pc+1)%16, )(b, b, out, (pc+1)%16, False, inp, program)
        case "IN", "A", None:
            return State(inp, b, out, (pc+1)%16, False, inp, program)
        case "MOV", "A", im:
            im_int = int(im)
            assert 0 <= im_int < 16
            return State(im_int, b, out, (pc+1)%16, False, inp, program)
        case "MOV", "B", "A":
            return State(a, a, out, (pc+1)%16, False, inp, program)
        case "ADD", "B", im:
            im_int = int(im)
            assert 0 <= im_int < 16
            b_tmp = b+im_int
            return State(a, b_tmp%16, out, (pc+1)%16, b_tmp>=16, inp, program)
        case "IN", "B", None:
            return State(a, inp, out, (pc+1)%16, False, inp, program)
        case "MOV", "B", im:
            im_int = int(im)
            assert 0 <= im_int < 16
            return State(a, im_int, out, (pc+1)%16, False, inp, program)
        case "OUT", "B", None:
            print(b)
            return State(a, b, b, (pc+1)%16, False, inp, program)
        case "OUT", im, None:
            im_int = int(im)
            assert 0 <= im_int < 16
            print(im_int)
            return State(a, b, im_int, (pc+1)%16, False, inp, program)
        case "JNC", im, None:
            im_int = int(im)
            assert 0 <= im_int < 16
            return State(a, b, out, im_int if not c else (pc+1)%16, False, inp, program)
        case "JMP", im, None:
            im_int = int(im)
            assert 0 <= im_int < 16
            return State(a, b, out, im_int, False, inp, program)
        case _:
            raise Exception("Unrecognized command")

# Bits and Bobbles


In [ ]:
%%file /tmp/td4.v
module TD4 (
    input clk,
    input rst,
    input [7:0] dip_switch [16],
    input [3:0] in_dip_switch,
    output reg [3:0] out_led,
    output reg [3:0] A,       // Added output for register A
    output reg [3:0] B,       // Added output for register B
    output reg [3:0] PC,      // Added output for program counter
    output reg carry          // Added output for carry flag
);

    // Instruction opcodes
    parameter ADD_A_IM = 4'b0000,  MOV_A_B = 4'b0001,   IN_A = 4'b0010, MOV_A_IM = 4'b0011;
    parameter MOV_B_A =  4'b0100, ADD_B_IM = 4'b0101,   IN_B = 4'b0110, MOV_B_IM = 4'b0111;
    parameter OUT_B =    4'b1001,   OUT_IM = 4'b1011, JNC_IM = 4'b1110,   JMP_IM = 4'b1111;

    // Main CPU cycle
    always @(posedge clk or posedge rst) begin
        if (rst) {A, B, PC, carry, out_led} <= 0;
        else begin
            // Data
            case (dip_switch[PC][7:4])
                ADD_A_IM:  {carry, A} <= A + dip_switch[PC][3:0];
                MOV_A_B:   A <= B;
                IN_A, MOV_A_IM:  A <= dip_switch;
                MOV_B_A:   B <= A;
                ADD_B_IM:  {carry, B} <= B + dip_switch[PC][3:0];
                IN_B, MOV_B_IM:  B <= dip_switch[PC][3:0];
                OUT_B:     out_led <= B;
                OUT_IM:    out_led <= dip_switch;
            endcase
            // Control
            case (dip_switch[PC][7:4])
                JNC_IM:    PC <= !carry ? dip_switch : PC + 1;
                JMP_IM:    PC <= dip_switch;
                default:   PC <= PC + 1;
            endcase
        end
    end
endmodule


In [ ]:
%%file /tmp/tb.v
module td4_tb;
    reg clk, rst;
    reg [3:0] dip_switch;
    wire [3:0] out_led;
    wire [3:0] A, B, PC;
    wire carry;

    // Instantiate the TD4 CPU with new outputs for A, B, PC, and carry
    TD4 td4 (
        .clk(clk), .rst(rst), .dip_switch(dip_switch),
        .out_led(out_led), .A(A), .B(B), .PC(PC), .carry(carry)
    );

    // Clock generation
    always #5 clk = ~clk;  // 10ns period (100MHz)

    initial begin
        // Initialize signals
        clk = 0;
        rst = 1;
        dip_switch = 4'b0000;

        // Release reset after some delay
        #20 rst = 0;

        // Test a simple sequence of instructions
        #10 dip_switch = 4'b0000;  // ADD A, Im
        #10 dip_switch = 4'b0001;  // MOV A, B
        #10 dip_switch = 4'b1001;  // OUT B
        #10 dip_switch = 4'b1111;  // JMP 0
        #10 dip_switch = 4'b0000;  // Set dip_switch to zero for testing

        // Run simulation for some cycles
        #1000 $finish;
    end

    // Monitor outputs
    initial begin
        $monitor("Time=%0t | clk=%b | rst=%b | dip_switch=%b | out_led=%b | A=%b | B=%b | PC=%b | carry=%b", 
                 $time, clk, rst, dip_switch, out_led, A, B, PC, carry);
    end
endmodule

In [ ]:
! iverilog -o /tmp/tb /tmp/td4.v /tmp/tb.v && vvp /tmp/tb